In [ ]:
pip install jointtsmodel

In [1]:
import os
cwd = os.getcwd()
cwd

'C:\\Users\\User\\jh2\\JST'

### 1. 감성사전 설정

In [53]:
# KNU 감성사전 기반 lexicon 생성 (긍정단어 - 1, 2점 -> 1점 // 부정단어 - -1, -2점 -> -1점 // 중립단어 제외)

import json
import pandas as pd

sentiment_df = pd.DataFrame(index=['Word','Sentiment'])
word_list = []
sentiment_list = []

with open('SentiWord_info.json', encoding='utf-8') as f:
    
    data = json.load(f)
    
    for i in data:
        root = i['word_root']
        score = int(i['polarity'])
        
        if score >= 1:
            score = 1
        
        elif score <= -1:
            score = -1
        
        elif score == 0:
            continue
        
        out = pd.DataFrame({'Word':[root], 'Sentiment':[score]})
        sentiment_df = pd.concat([sentiment_df, out])
    
    
sentiment_df = sentiment_df.reset_index(drop=True).dropna()


# 부정단어 추가
neg_word = ["모순", "고배", "대가리", "부패", "퇴출", "비판", "우려", "논란", "죽음", "비난", "손해", "지적", "어쩌고", "경고", "당장", "횡령",
            "경고", "외면", "저조", "사기", "힘들다", "침체", "위협", "미흡", "정체", "피해자", "재난", "미안", "차별", "불매", "재해", "우려",
            "참사", "워싱","그린워싱", "부채", "낭비", "전쟁", "무시", "취약", "붕괴", "사태", "위기", "약화", "걱정", "미흡", "현실", "공격"]
neg_score = [-1]*len(neg_word)


neg_df = pd.DataFrame(zip(neg_word, neg_score), columns=['Word', 'Sentiment'])



# 긍정단어 추가
pos_word = ['응원', "진심", '신뢰', '힘내', '상생', '인기', '공감', '호평', '기대'] 
pos_score = [1]*len(pos_word)
pos_df = pd.DataFrame(zip(pos_word, pos_score), columns=['Word', 'Sentiment'])


sentiment_df = pd.concat([sentiment_df, neg_df, pos_df], ignore_index=True)
sentiment_df.to_excel('prior_sentiment_kor.xlsx', encoding='utf-8-sig', index=False)

C:\Users\User\anaconda3\envs\jh2\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
neg_df

### 2.전처리함수 설정

In [38]:
import re
import pandas as pd
import openpyxl
import os
import nltk

#visualization
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

from konlpy.tag import Mecab, Kkma, Okt
from iteration_utilities import unique_everseen
from wordcloud import WordCloud
from matplotlib import font_manager, rc
from bs4 import BeautifulSoup



path_list = ['C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/블로그/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/유튜브/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/인스타/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/트위터/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/블로그/2022',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/유튜브/2022/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/인스타/2022',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/트위터/2022/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/보고서/']


def file_process(n, dt='xlsx'):
    # 경로설정
    path = path_list[n]

    # 결과 추출을 위한 이름 설정
    name = set_name(path)

    df = open_file(path, datatype=dt)

    tqdm.pandas(desc="Tokenization Progress")
    df['tokens'] = df['text'].progress_apply(preprocess, custom_dict=custom_dict)
    df['text'] = df['text'].progress_apply(pre_process)


    # 중복되는 df 제거
    df.drop_duplicates(subset=['text'], keep='first', inplace=True)
    df['num_tokens'] = df['tokens'].map(len)

    # 토큰값이 0인 행 제거
    df = df.loc[df['num_tokens'] != 0]

    # index 초기화
    df.reset_index(drop=True, inplace=True)
    
    return df

"""

파일 불러오기를 위한 함수 설정 

- path: 경로
- datatype= 'docx' or 'xlsx'
- min_text_docx = n 문단최소글자수 (default == 5)
- min_text_xlsx = n 엑셀 최소 글자수 (default == 5)

"""

def open_file(path, datatype=None, min_text_docx = 5, min_text_xlsx = 5):
    pd.set_option('display.max_colwidth', None)
    files_xlsx = []
    files_docx = []

    if datatype == 'docx':
        files_docx = [file for file in os.listdir(path) if file.endswith('.docx')]
        paragraphs = []
        titles = []
        
        for i in range(len(files_docx)):
            file_name = path+files_docx[i]
            text = docx2txt.process(file_name)
            doc_paragraphs = [p.strip() for p in text.split('\n'*3) if len(p.strip()) > min_text_docx]
            # doc_paragraphs = [p.strip() for p in re.split('\n+', text) if len(p.strip()) > min_text_docx]            
            
            titles += [files_docx[i]] * len(doc_paragraphs)
            paragraphs += doc_paragraphs
            
        df = pd.DataFrame(paragraphs, columns=['text'])
        # df = pd.DataFrame(list(zip(titles, paragraphs)), columns=['title', 'text'])

    elif datatype == 'xlsx':
        files_xlsx = [file for file in os.listdir(path) if file.endswith('.xlsx')]
        df = pd.DataFrame()
        
        if '유튜브' in path:
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                
                df_title = pd.read_excel(file_path, usecols=['제목'])
                df_title = df_title.rename(columns={'제목': 'text'})   
                df_title = df_title.drop_duplicates(keep='first')

                df_text = pd.read_excel(file_path, usecols=['댓글'])
                df_text = df_text.rename(columns={'댓글': 'text'})   
                
                df = pd.concat([df, df_title, df_text], ignore_index=True)
                
#         elif '인스타' in path:
#             for f in files_xlsx:
#                 file_path = os.path.join(path, f)
                
#                 df_text = pd.read_excel(file_path, usecols=['본문'])
#                 df_text = df_text.rename(columns={'본문': 'text'})   
#                 df_text = df_text.drop_duplicates(keep='first')

#                 df_com = pd.read_excel(file_path, usecols=['댓글'])
#                 df_com = df_com.rename(columns={'댓글': 'text'})  
                
                
#                 df = pd.concat([df, df_text, df_com], ignore_index=True)

        # 댓글 각자 문서로 처리
    
        elif '인스타' in path:
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                df_text = pd.read_excel(file_path, usecols=['본문'])
                df_text = df_text.rename(columns={'본문': 'text'})
                df_text = df_text.drop_duplicates(keep='first')
                df_com = pd.read_excel(file_path, usecols=['댓글'])
                df_com = df_com.rename(columns={'댓글': 'text'})
                
                # 각 댓글을 하나의 문서로 분리
                df_com['text'] = df_com['text'].str.split('/')
                df_com = df_com.explode('text')
                
                df = pd.concat([df, df_text, df_com], ignore_index=True)

        elif '트위터' in path:
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                df_read = pd.read_excel(file_path, usecols=['text'])
                df = pd.concat([df, df_read])

        elif '블로그' in path:
            
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                
                # df_title = pd.read_excel(file_path, usecols=['title'])
                # df_title = df_title.rename(columns={'title': 'text'})   
                # df_title = df_title.drop_duplicates(keep='first')

                df_text = pd.read_excel(file_path, usecols=['text'])
                df = pd.concat([df, df_text], ignore_index=True)

                # df = pd.concat([df, df_title, df_text], ignore_index=True)
                

    return df.dropna()

# 형태소 분석을 위한 객체 생성
mecab = Mecab(dicpath='c:/mecab/mecab-ko-dic')

# 1차 전처리 (한글 이외에 모든 텍스트 제거)
def pre_process(text):
    
    pr_text = re.sub(r"[^ㄱ-ㅣ가-힣\s]+|[ㄱ-ㅎㅏ-ㅣ]+", "", text)
    pr_text = re.sub('\\<br\\>',' ',pr_text)
    pr_text = re.sub(r'\s+', ' ', pr_text, flags=re.I)
    pr_text = pr_text.replace("\n", "")
    pr_text = pr_text.strip()
    return pr_text

# 2차 전처리 mecab을 통한 형태소 분석
def analyzer_pos(text, custom_dict):
    tokens = mecab.pos(text)
    # tokens = [ s for s, t in tokens if t in ['XR', 'VV', 'VA', 'NNG', 'NNP'] and len(s) > 1 and s not in stop_word] 
    # tokens = [ s for s, t in tokens if len(s) > 1 and s not in stop_word] # 전체 품사 중 3글자 이상 + 불용어를 제외함
    tokens = [ s for s, t in tokens if t in ['VV', 'VA', 'NNG', 'NNP'] and len(s) > 1 and s not in stop_word] # 동사, 형용사, 명사 중 2글자 이상 + 불용어를 제외함

    # 원하는 단어가 끊여져서 나올경우(우선순위 설정 이후) 합치기
    i = 0
    new_tokens = []

    while i < len(tokens):
        if i == len(tokens) - 1:
            new_tokens.append(tokens[i])
            break
        current_token = tokens[i]
        next_token = tokens[i+1]
        if current_token+next_token in custom_dict:
            new_tokens.append(current_token+next_token)         
            i += 2
        else:
            new_tokens.append(current_token)
            i += 1
    return new_tokens


def set_name(path):
    name = ''
    
    if '보고서' in path:
        name = '보고서'

    elif '인스타' in path:
        name = '인스타'

    elif '블로그' in path:
        name = '블로그'

    elif '트위터' in path:
        name = '트위터'

    elif '유튜브' in path:
        name = '유튜브'
    
    return name


"""
텍스트를 입력받아 사용자 정의 사전을 기반으로 정규표현식 기반 전처리, mecab 형태소 분석을 진행

"""

def preprocess(text, custom_dict=None):
    
    # 정규표현식 처리
    text = pre_process(text)
    
    # mecab 전처리
    tokens = analyzer_pos(text, custom_dict)
    # tokens = ", ".join(tokens)

    return tokens

### 3. 불용어사전 호출

In [ ]:
# 불용어
# 길호현. (2018). 텍스트마이닝을 위한 한국어 불용어 목록 연구. 우리말글, 78, 1-25.

with open("stop.txt", "r", encoding="utf8") as f:
    stop_word = []
    stop_text = [stop_word.extend(line.strip().split(", ")) for line in f]

    
# with open("stop2.txt", "r", encoding="utf8") as f:
#     stop_word2 = []
#     stop_text2 = [stop_word2.extend(line.strip().split(", ")) for line in f]
# print(stop_word2)

# 개인 전처리를 위한 stop_word 설정
stop_word_jh = ['최태원', '디스클로저', '외신', '출처', '한국', '뉴스', '최초', '공개', '단독', '회사', '상장', '보도', '글로벌', '발행', '래퍼', '출연', '리뷰', '중앙일보', '기자', '이낙연',
                '디렉', '명상', '회장', '신문', '나라', '블로그', '코리아', '미국', '중국', '관련', '대한민국', '대리석', '타임스', '무디스', '법무법인', '변호사', '교수', '기업',
                '타임즈', '인더스', '트리', '습니다', '에서', '속도', '한다', '이승기', '방위대', '일본', '는데', '으로', '처럼', '호남', '야구', '위한', '면서', '지만', '에게', '천억', '투데이', '해요', '펭귄', '팝콘',
                '북마크', '네이버', '네슬레', '자칭', '네요', '코엑스', '노스페이스', '홈페이지', '어야', '전라', '전북', '매일신문', '감사', '영상', '만들', '방송', '슈카', '오늘', '배우', '세상', '사람',
                '다니', '애플', '나오', '삼전', '민주당', '선생', '선수', '이번', '마찬가지', '그동안', '동원', '참치', '맛있', '한화', '그룹', '문재인', '래원', '노래', '좌파', '휠라', '기판',
                '하이닉스', '한미', '입니다', '게임', '넷마블', '삼성', '시멘트', '조선', '백화점', '약품', '의약품', '해양', '효성', '렌탈', '울산', '자동차', '그램', '정답',
                '빅이슈', '삼양', '두부', '부면', '포스코', '하림', '비빔면', '상공', '도시락', '스타', '미주', '라무', '유한', '킴벌리', '농협', '모델', '프로', '아거', '연지', '김병만',
                '떨채', '리비', '프로', '퀴즈', '경기주택공사', '경기', '주택', '공사', '진분', '테슬라', '교보생명', '푸르', '서울', '공단', '삼성전자', '부문', '카카오', '데일리', '임팩트',
                '국내', '시사', '센터', '앨범', '지디', '엔터', '부산', '연구원', '이벤트', '출장', '박스', '안치용', '슈가버블', '배민', '산불', '피해', '클릭', '지수', '하이브', '호비', '경기도',
                '래리', '핑크', '기사', '링크', '윤석열', '생각', '케어', '오전', '오후', '제주', '가능', '내용', '경우', '정도', '소개', '대표', '언서', '학교', '대학교', '학생', '사진', '사용', '현대',
                '원문']

stop_word_jh2 = ['토끼', '트레저', '센트', '파리바', '마을금고', '하이트진로', '꿀벌', '부영', '전남', '아이돌', '주니어', '로봇', '요약', '사단법인', '돌아가', '멜론', '한눈', '버핏', '셋톱박스',
                  '우편물', '현태캐피탈,', '목소리', '인사이드', '포유', '여대', '롯데제과', '김치', '대하', '홍은택', '동네', '소프트', '윤종규', '산업은행', '윤종규', '동부', '한화투자증권', '머리',
                  '이러', '조현준', '썸머', '구내식당', '포항,', '유한양행', '제클린', '국민카드', '대우건설', '구미', '스타일', '박옥', '에버', '부천', '랜드', '프리', '선대', '아산', '한국지역난방',
                  '테일', '이미주', '디자인', '풀무원', '매경', '롯데케미칼', '컬러,', '현대로템', '포항시', '남형', '현산', '아뉴스', '중기', '뉴스룸', '인더', '전시', '부산시', '일보', '국민은행',
                  '성동구', '에듀', '동안', '민회', '대선', '적십자사', '수협', '진원', '파크', '한화건설', '텔레콤,', '지사', '데이', '크리에이터', '해남', '공원', '유머', '여수', '롯데카드', '케이',
                  '특집', '현대제철', '모건', '글라스', '마사회', '인덱스', '아시아나항공', '텍스', '리더스', '파이낸셜', '스테이', '두산,', '하우', '전주시', '한국거래소', '국마', '계룡', '대한상의',
                  '골드', '신보', '항만', '미스트', '인천공항공사', '포카', '한경', '우리금융그룹', '단백질', '축구', '제일제당', '트위터', '주세요', '파우,', '건국대', '코카', '이주현', '최중현', '중부',
                  '퍼시픽', '연합회', '슈나이더', '드리', '코노', '아모레', '청인', '손잡', '해남군', '원장', '안랩', '로직스', '알리', '산학', '버튼', '노컷뉴스', '워크', '수력', '이제', '포스코건설,',
                  '다회', '시립', '월드', '홍보', '박사', '강동완', '태승', '이타임즈', '강원도', '코람코', '석화', '신용보증기금', '남양주', '데모', '독스', '헬스', '용가리동아', '인수위', '그러', '이마트',
                  '일렉트릭', '조원', '은행장,', '위아', '알아보', '코레', '홈플러스', '바디스', '이윤진', '닷컴', '위메이드', '전경련', '맥주', '고은정', '스리랑카', '그렇', '신지현', '이노', '끝내',
                  '캠프', '회차', '안동', '레포트', '지오', '헤럴드', '유업', '이카', '진천', '단심', '유업', '이카','피플', '고배', '이카', '출판사', '퇴계원', '엡손', '광명시', '난지', '아줌마', '주간경향', '버티',
                  '달수', '아빠', '언론사', '특별시', '차기', '대상', '브레이크', '제회', '플러스', '후계자', '골프', '연예', '인천', '무료', '러시아', '의원', '수원', '서부', '이종현', '준원', '케미',
                  '지회', '강자', '남양주시', '비즈', '손해', '연합뉴스', '김주현', '피니언', '실시간', '스포츠']

stop_word.extend(stop_word_jh)
stop_word.extend(stop_word_jh2)

print(stop_word)
print(len(stop_word))

custom_dict = ['지속가능', '이해관계자', '탄소중립', '신재생에너지', '재생에너지', '탄소중립', '탄소제로', '메타버스', '지배구조', '지속가능보고서', '지속가능경영보고서', '사외이사',
               '이상기후', '기후변화', '게임체인저', '동반성장', '고부가가치', '협력사', '가치사슬', '밸류체인', '유연근무', '정보보호', '이행원칙', '노사협의회', '위드코로나', '제로웨이스트',
               '제로웨이스트', '위드코로나', '암호화폐', '그린워싱', '배당귀족']


### 4. 데이터 호출 및 전처리

In [72]:
import pandas as pd
import os
import docx2txt
from tqdm import tqdm 
tqdm.pandas()


""" 
path_list (블로그/유튜브/인스타/트위터) + 보고서
[0~3] - 2020/2021  0:블로그 / 1: 유튜브 / 2: 인스타 / 3: 트위터
[4~7] - 2022       4:블로그 / 5: 유튜브 / 6: 인스타 / 7: 트위터
[8]   - 보고서      8: 보고서

"""

df = pd.DataFrame()
# df = file_process(4)

# df_blog = file_process(4)
# df_yt = file_process(5)
# df_insta = file_process(6)
# df_twitter = file_process(7)

df = file_process(7)

# df = pd.concat([df, df_blog, df_yt, df_insta, df_twitter], ignore_index=True)
df.to_excel('소셜미디어_전처리.xlsx', encoding='utf-8-sig')

Tokenization Progress: 100%|█████████████████████████████████████████████████| 21001/21001 [00:00<00:00, 121095.10it/s]
C:\Users\User\anaconda3\envs\jh2\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
# 단어사전 확인
with open("C:\\mecab\\user-dic\\nnp.csv", 'r', encoding='utf-8') as f: 
    file_data = f.readlines()
file_data

In [ ]:
# 우선순위 확인
with open("c:/mecab/mecab-ko-dic/user-nnp.csv", 'r', encoding='utf-8') as f: 
    file_data = f.readlines()
file_data

In [19]:
df['tokens'].isna().sum()

0

In [79]:
import pandas  as pd
import ast

"""
데이터 호출 & 전처리 결과 호출

"""

df= pd.read_excel('소셜미디어_전처리.xlsx', dtype={'tokens': list})
df['tokens'] = df['tokens'].apply(lambda x: ast.literal_eval(x))

### 4.1. 토큰 정리

In [80]:
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('모르', '모르다') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('줄이', '줄이다') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('재밌', '재밌다') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('밸류체인', '가치사슬') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('어쩌고', '어쩌구') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('어떻', '어떻게') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('힘드', '힘들다') for i in x])
df['tokens'] = df['tokens'].apply(lambda x: [i.replace('만드', '만들다') for i in x])

# 대상 단어들
da_list = ['모르', '빠르', '재밌', '미치', '대하', '보내', '바라보', '떨어지', '떠오르', '지키', '힘들', '모르', '맞추', '이끌', '구하', '낮추', '없애', '따지', '애쓰', '커지', '늘리', '키우',
           '살리', '앞두']

# 데이터프레임의 'tokens' 열에 적용할 함수
def replace_word_with_da(word):
    if word in da_list:
        return word + '다'
    else:
        return word

# 데이터프레임에 함수 적용하기
df['tokens'] = df['tokens'].apply(lambda x: [replace_word_with_da(word) for word in x])

### 5.JST

In [81]:
from jointtsmodel.RJST import RJST
from jointtsmodel.JST import JST
from jointtsmodel.sLDA import sLDA
from jointtsmodel.TSM import TSM
from jointtsmodel.TSWE import TSWE

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from jointtsmodel.utils import *


text = df['tokens'].apply(lambda x: ' '.join(x))
vectorizer = CountVectorizer(max_df=0.5,
                             min_df=3,
                             # ngram_range=(1,2),          #바이그램 사용시 성능 확인
                             
                             max_features=3000)

X = vectorizer.fit_transform(text)
vocabulary = vectorizer.get_feature_names_out()

inv_vocabulary = dict(zip(vocabulary,np.arange(len(vocabulary))))
lexicon_data = pd.read_excel('./prior_sentiment_kor.xlsx')
lexicon_data = lexicon_data.dropna()
lexicon_dict = dict(zip(lexicon_data['Word'],lexicon_data['Sentiment']))

In [84]:

# for n in range(3, 16):

model = RJST(n_topic_components=3, n_sentiment_components=2, random_state=123,evaluate_every=10, max_iter=30)
model.fit(X.toarray(), lexicon_dict)

# model.transform()[:2]

top_words = list(model.getTopKWords(vocabulary).values())

'''
utils.py // def coherence_score_umass() 깃허브 확인 후 코드 수정 (127~129) 2023-03-19

'''
# coherence_score_umass(X.toarray(),inv_vocabulary,top_words)
coherence_score_uci(X.toarray(),inv_vocabulary,top_words)           

Perplexity after iteration 10 (out of 30 iterations) is 2474.03
Perplexity after iteration 20 (out of 30 iterations) is 2033.29
Perplexity after iteration 30 (out of 30 iterations) is 1951.69


0.20357482701902432

In [85]:
model.getTopKWords(ㅊ, num_words=10)

{(1, 1): ['경영', '실천', '사회', '개최', '지원', '위원회', '사업', '활동', '수상', '지역'],
 (1, 2): ['투자', '사회', '세계', '에너지', '펀드', '공시', '시장', '시대', '중요', '주식'],
 (2, 1): ['경영', '금융', '은행', '진행', '공모전', '발표', '경제', '포럼', '모집', '전략'],
 (2, 2): ['경영', '평가', '강화', '등급', '발간', '금융', '기관', '획득', '건설', '보고서'],
 (3, 1): ['경영', '테크', '트렌드', '협약', '지원', '체결', '혁신', '중소기업', '업무', '스타트업'],
 (3, 2): ['환경', '친환경', '경영', '사회', '탄소', '캠페인', '탄소중립', '기후', '경제', '활용']}

In [ ]:
# 수정사항 확인용
import inspect
print(inspect.getsource(coherence_score_umass))

In [12]:
model.perplexity()

1651.2570725141406

In [185]:
top_words = list(model.getTopKWords(vocabulary).values())